In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps
from FinMind.data import DataLoader
import helper
api = DataLoader()

d:\Practice\stock_trade\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def print_result(stats):
  stats = stats.loc[[
      'Start',
      'End',
      'Equity Final [$]',      # 最終資產淨值
      'Return [%]',            # 總報酬率
      'Buy & Hold Return [%]', # 買入並持有報酬率（基準）
      'Max. Drawdown [%]',     # 最大回撤
      '# Trades',
      'Win Rate [%]',          # 勝率
      'SQN',                    # 系統品質數字
      '_trades',
      '_strategy'
    ]]
  print('------------------回測表現-----------------')
  print(stats)
  print('------------------交易紀錄-----------------')
  trades = stats['_trades']
  for _, trade in trades.iterrows():
      print(f"Buy date: {trade['EntryTime']}, Sell date: {trade['ExitTime']}")
      print(f"Buy price: {trade['EntryPrice']}, Sell price: {trade['ExitPrice']}, P/L: {trade['ReturnPct']:.2%}")

In [3]:
today = datetime.today() #datetime.strptime('2025-04-30', "%Y-%m-%d")
start_date = (today - relativedelta(months=24)).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")
print('start date', start_date)
print('end date', end_date)

ticker = '2317'
df = api.taiwan_stock_daily(
    stock_id=ticker,
    start_date=start_date,
    end_date=end_date
)
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df = df.rename(columns={'max': 'High', 'min': 'Low', 'open': 'Open', 'close': 'Close', 'Trading_Volume': 'Volume'})
df

2025-11-12 09:34:32.925 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanStockPrice, data_id: 2317


start date 2023-11-12
end date 2025-11-12


,stock_id,Volume,Trading_money,Open,High,Low,Close,spread,Trading_turnover
date,,,,,,,,,
2023-11-13,2317,18461159,1806009405,97.8,98.4,97.5,97.7,0.5,13027
2023-11-14,2317,47364152,4734302962,98.4,101.0,98.2,100.0,2.3,27140
2023-11-15,2317,51481834,5164217515,100.5,101.5,99.8,100.5,0.5,26562
2023-11-16,2317,31965062,3226643998,101.0,101.5,100.5,101.0,0.5,14473
2023-11-17,2317,47314771,4852922519,101.5,103.5,101.0,102.5,1.5,23755
...,...,...,...,...,...,...,...,...,...
2025-11-05,2317,114664242,27783011433,239.0,248.5,237.0,247.0,3.0,102625
2025-11-06,2317,65794425,16345639683,249.5,250.5,246.0,248.0,1.0,54035
2025-11-07,2317,50319199,12385551288,246.0,250.5,244.0,244.0,-4.0,55005


In [4]:
# helper.EMA_KD.n1 = 20
# helper.EMA_KD.n2 = 60
bt = Backtest(df, helper.EMA_KD, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[20, 50, 60, 100, 120],
  kd=80,
  maximize='Equity Final [$]'
)
# stats = bt.run()
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/20 [00:00<?, ?it/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:168: RuntimeWarning: divide by zero encountered in divide
  x = value / self._data.Close
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:168: RuntimeWarning: divide by zero encountered in divide
  x = value / self._data.Close

Backtest.run:   0%|          | 0/476 [00:00<?, ?bar/s]d

------------------回測表現-----------------
Start                                                  2023-11-13 00:00:00
End                                                    2025-11-11 00:00:00
Equity Final [$]                                                169611.884
Return [%]                                                       69.611884
Buy & Hold Return [%]                                           142.857143
Max. Drawdown [%]                                                -4.889901
# Trades                                                                 6
Win Rate [%]                                                         100.0
SQN                                                               2.796807
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                         EMA_KD(n1=5,n2=20,kd=80)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-01-29 00:00:00, Sell date: 2024-03-18 00:00:00
Buy price: 102.5, S

In [5]:
helper.EMA_VWAP_KD.n1 = 10
helper.EMA_VWAP_KD.n2 = 50
bt = Backtest(df, helper.EMA_VWAP_KD, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[10, 20, 50, 60, 100, 120],
  maximize='Equity Final [$]'
)
# stats = bt.run()
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/24 [00:00<?, ?it/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=163: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close t

------------------回測表現-----------------
Start                                                  2023-10-30 00:00:00
End                                                    2025-10-28 00:00:00
Equity Final [$]                                              96856.661175
Return [%]                                                       -3.143339
Buy & Hold Return [%]                                                  0.0
Max. Drawdown [%]                                                -5.331868
# Trades                                                                 1
Win Rate [%]                                                           0.0
SQN                                                                    NaN
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                        EMA_VWAP_KD(n1=20,n2=100)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-12-26 00:00:00, Sell date: 2024-12-27 00:00:00
Buy price: 244.0, S

In [5]:
# helper.SmaCross.n1 = 10
# helper.SmaCross.n2 = 100
bt = Backtest(df, helper.SmaCross, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[10, 20, 50, 60, 100, 120],
  maximize='Equity Final [$]'
)
# stats = bt.run()
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/24 [00:00<?, ?it/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:168: RuntimeWarning: divide by zero encountered in divide
  x = value / self._data.Close


d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:168: RuntimeWarning: divide by zero encountered in divide
  x = value / self._data.Close
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:168: RuntimeWarning: divide by zero encountered in divide
  x = value / self._data.Close
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=52: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=65: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=140: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=184: Brok

------------------回測表現-----------------
Start                                                  2023-11-13 00:00:00
End                                                    2025-11-11 00:00:00
Equity Final [$]                                             273178.203113
Return [%]                                                      173.178203
Buy & Hold Return [%]                                           142.857143
Max. Drawdown [%]                                               -99.974269
# Trades                                                                 2
Win Rate [%]                                                         100.0
SQN                                                               1.302876
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                             SmaCross(n1=5,n2=60)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-02-21 00:00:00, Sell date: 2024-08-06 00:00:00
Buy price: 103.5, S

In [5]:
bt = Backtest(df, helper.SMA_KD, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[10, 20, 50, 60, 100, 120],
  kd=80,
  maximize='Equity Final [$]'
)
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/24 [00:00<?, ?it/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=43: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=53: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Pract

------------------回測表現-----------------
Start                                                  2023-10-30 00:00:00
End                                                    2025-10-28 00:00:00
Equity Final [$]                                             128471.594185
Return [%]                                                       28.471594
Buy & Hold Return [%]                                            -2.749141
Max. Drawdown [%]                                               -58.083147
# Trades                                                                 3
Win Rate [%]                                                         100.0
SQN                                                               3.206065
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                         SMA_KD(n1=5,n2=20,kd=80)
dtype: object
------------------交易紀錄-----------------
Buy date: 2023-12-28 00:00:00, Sell date: 2024-03-14 00:00:00
Buy price: 92.8, Se

In [6]:
helper.BOLL_KD30.buy_strategy = helper.BuyStrategy.BOLL_KD30
bt = Backtest(df, helper.BOLL_KD30, cash=100_000, commission=.001425)
stats = bt.run()
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:168: RuntimeWarning: divide by zero encountered in divide
  x = value / self._data.Close
Backtest.run:   0%|          | 0/465 [00:00<?, ?bar/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=176: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=177: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=253: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=266: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.v

------------------回測表現-----------------
Start                                                  2023-11-13 00:00:00
End                                                    2025-11-11 00:00:00
Equity Final [$]                                               80126.39455
Return [%]                                                      -19.873605
Buy & Hold Return [%]                                           142.857143
Max. Drawdown [%]                                               -48.858241
# Trades                                                                 1
Win Rate [%]                                                           0.0
SQN                                                                    NaN
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                                        BOLL_KD30
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-07-23 00:00:00, Sell date: 2025-05-13 00:00:00
Buy price: 198.5, S

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\_stats.py:157: RuntimeWarning: divide by zero encountered in divide
  market_log_returns = np.log(c[1:] / c[:-1])
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\_stats.py:157: RuntimeWarning: divide by zero encountered in log
  market_log_returns = np.log(c[1:] / c[:-1])
